__상관계수 : ex) 아빠&자식 키 관계__  

인덱스 기준: sort_index  
데이터 기준: sort_values  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
obj = pd.Series(range(4),index=['j','i','k','p'])

In [5]:
obj.sort_index()

i    1
j    0
k    2
p    3
dtype: int64

In [7]:
obj.sort_values(ascending=False)

p    3
k    2
i    1
j    0
dtype: int64

In [11]:
df=pd.DataFrame(np.arange(8).reshape((2,4)),
            index=['three','one'],
            columns=['k','j','i','p'])
df.sort_index()

,k,j,i,p
one,4,5,6,7
three,0,1,2,3


In [13]:
df.sort_index(axis=1,ascending=False)

,p,k,j,i
three,3,0,1,2
one,7,4,5,6


In [15]:
obj=pd.Series([2,np.nan,5,-3,0])
obj.sort_values()

3   -3.0
4    0.0
0    2.0
2    5.0
1    NaN
dtype: float64

In [16]:
df2=pd.DataFrame({'z':[3,2,-1,5],'a':[0,1,0,1]})
df2

,z,a
0,3,0
1,2,1
2,-1,0
3,5,1


In [18]:
#df2.sort_values() # Error - 데이타 프레임은 어떤 시리즈(컬럼) 기준으로 정렬할지를 정해 주어야 오류가 나지 않음
df2.sort_values(by='a')

,z,a
0,3,0
2,-1,0
1,2,1
3,5,1


In [19]:
# 1차, 2차 정렬 설정하기

df2.sort_values(by=['a','z'])

,z,a
2,-1,0
0,3,0
1,2,1
3,5,1


In [20]:
obj2=pd.Series(range(5),index=['a','a','b','c','c'])
obj2

a    0
a    1
b    2
c    3
c    4
dtype: int64

In [25]:
obj2.index.unique()

Index(['a', 'b', 'c'], dtype='object')

In [27]:
df3=pd.DataFrame(np.random.randn(4,3),index=['a','a','b','b'])
df3

,0,1,2
a,1.064388,0.766013,-1.468812
a,0.797722,1.546922,-0.641691
b,1.375397,-0.077565,-1.962610
b,1.231066,0.080388,0.619860


In [32]:
df3.loc['b']

,0,1,2
b,1.375397,-0.077565,-1.96261
b,1.231066,0.080388,0.61986


__피어슨 상관계수 기반 추천시스템__

In [34]:
df=pd.DataFrame([[1.3,np.nan],
             [7.5,-5.5],
             [np.nan,np.nan],
             [0.5,-1.5]],
               index=['a','b','c','d'],
               columns=['one','two'])
df

,one,two
a,1.3,NaN
b,7.5,-5.5
c,NaN,NaN
d,0.5,-1.5


In [35]:
df.sum()

one    9.3
two   -7.0
dtype: float64

In [37]:
df.cumsum()

,one,two
a,1.3,NaN
b,8.8,-5.5
c,NaN,NaN
d,9.3,-7.0


In [38]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.100000,-3.500000
std,3.831449,2.828427
min,0.500000,-5.500000
25%,0.900000,-4.500000
50%,1.300000,-3.500000
75%,4.400000,-2.500000
max,7.500000,-1.500000


In [40]:
S = pd.Series(['a','a','b','b','c']*3)
S.describe()

count     15
unique     3
top        b
freq       6
dtype: object

__협업 필터링 알고리즘 기반 추천시스템 제작__

In [41]:
critics={
    'BTS':{'암수살인':5, '바울':4, '할로윈':1.5},
    '손흥민':{'바울':5, '할로윈':2},
    '레드벨벳':{'암수살인':2.5, '바울':2, '할로윈':1},
    '트와이스':{'암수살인':3.5, '바울':4, '할로윈':5}
}

임의 두 사람간 유사도 --> 예상 평점

__협업 필터링__  
* User-based : 사용자 기반 CF  
* item-based : 아이템 기반 CF  
* hybrid-based : 혼합 CF  

In [43]:
critics['손흥민']['할로윈']

2

In [45]:
from math import sqrt

In [46]:
def sim(i,j):
    return sqrt(pow(i,2)+pow(j,2))

In [47]:
# 손흥민과 트와이스 거리 계산.
delta_x = critics['손흥민']['바울']-critics['트와이스']['바울']
delta_y = critics['손흥민']['할로윈']-critics['트와이스']['할로윈']
D_st = sim(delta_x,delta_y)
D_st

3.1622776601683795

In [48]:
def sim_2(L):
    ret=0
    for element in L:
        ret+=pow(element,2)
    return sqrt(ret)

In [54]:
piv='손흥민'
for key in critics:
    if key!=piv:
        L=[]
        for sub_key in critics[key]:
            if sub_key in critics[piv].keys():
                L.append(critics[piv][sub_key]-critics[key][sub_key])
        dist=sim_2(L)
        print(f'{piv} 와 {key} : {1/(dist+1)}')

손흥민 와 BTS : 0.4721359549995794
손흥민 와 레드벨벳 : 0.2402530733520421
손흥민 와 트와이스 : 0.2402530733520421


In [51]:
# 정규화 : 각 열을 기준으로 데이터 값을 0~1 사이에 mapping 되도록
# data = data - data_min / data_max - data_min

In [61]:
c_df = pd.DataFrame(critics)
c_df

,BTS,손흥민,레드벨벳,트와이스
암수살인,5.0,NaN,2.5,3.5
바울,4.0,5.0,2.0,4.0
할로윈,1.5,2.0,1.0,5.0


In [62]:
c_df = (c_df-c_df.min()) / (c_df.max()-c_df.min())
c_df

,BTS,손흥민,레드벨벳,트와이스
암수살인,1.000000,NaN,1.000000,0.000000
바울,0.714286,1.0,0.666667,0.333333
할로윈,0.000000,0.0,0.000000,1.000000


In [63]:
critics2 = {

    '레드벨벳': {'택시운전사': 2.5,'겨울왕국': 3.5,
             '리빙라스베가스': 3.0,'넘버3': 3.5,
             '사랑과전쟁': 2.5,'세계대전': 3.0},

    'BTS': {'택시운전사': 1.0,'겨울왕국': 4.5,
            '리빙라스베가스': 0.5,'넘버3': 1.5,
            '사랑과전쟁': 4.5,'세계대전': 5.0},

    '블랙핑크': {'택시운전사': 3.0,'겨울왕국': 3.5,
             '리빙라스베가스': 1.5,'넘버3': 5.0,
             '세계대전': 3.0,'사랑과전쟁': 3.5},

    '소녀시대': {'택시운전사': 2.5,'겨울왕국': 3.0,
             '넘버3': 3.5,'세계대전': 4.0},

    '마마무': {'겨울왕국': 3.5,'리빙라스베가스': 3.0,
            '세계대전': 4.5,'넘버3': 4.0,
            '사랑과전쟁': 2.5},

    '오마이걸': {'택시운전사': 3.0,'겨울왕국': 4.0,
             '리빙라스베가스': 2.0,'넘버3': 3.0,
             '세계대전': 3.5,'사랑과전쟁': 2.0},

    '모모랜드': {'택시운전사': 3.0,'겨울왕국': 4.0,
             '세계대전': 3.0,'넘버3': 5.0,
             '사랑과전쟁': 3.5},

    '우주소녀': {'겨울왕국': 4.5, '사랑과전쟁': 1.0,
             '넘버3': 4.0}

}